In [4]:
import os
import re
import fitz 
import docx  

In [48]:
try:
    import win32com.client
except ImportError:
    win32com = None

In [7]:
def extract_text_from_pdf(pdf_path):
    """Trích xuất toàn bộ văn bản từ file PDF."""
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text("text") + "\n"
    return text


In [50]:
def extract_text_from_doc(doc_path):
    """
    Trích xuất văn bản từ file DOC (.doc) sử dụng COM automation của MS Word.
    Chỉ sử dụng trên Windows khi đã cài Microsoft Word và pywin32.
    """
    if win32com is None:
        raise ImportError("win32com module not found. Vui lòng cài đặt pywin32 để xử lý file .doc.")
    
    word = win32com.client.Dispatch("Word.Application")
    word.Visible = False
    abs_path = os.path.abspath(doc_path)
    try:
        doc = word.Documents.Open(abs_path)
        text = doc.Range().Text
        doc.Close(False)
    except Exception as e:
        print(f"Lỗi khi xử lý {doc_path}: {e}")
        text = ""
    finally:
        word.Quit()
    return text


In [54]:
def sanitize_name(name):
    """
    Loại bỏ các ký tự không hợp lệ cho tên file/folder 
    và loại bỏ khoảng trắng thừa.
    """
    return re.sub(r'[\\/*?:"<>|]', "", name).strip()

def prepare_filename(name, max_length=150):
    """
    Chuẩn hóa tên: loại bỏ ký tự không hợp lệ và rút gọn nếu độ dài vượt quá max_length.
    """
    name = sanitize_name(name)
    if len(name) > max_length:
        name = name[:max_length]
    return name

In [8]:
def extract_text_from_docx(docx_path):
    """Trích xuất toàn bộ văn bản từ file DOCX."""
    text = ""
    doc = docx.Document(docx_path)
    for para in doc.paragraphs:
        text += para.text + "\n"
    return text


In [56]:
def split_into_chapters(text):
    """
    Tách văn bản thành các chương dựa trên marker "Chương ...".
    Nếu không tìm thấy marker, toàn bộ nội dung sẽ được gán vào "Chương 1".
    """
    pattern = re.compile(r"^(Chương\s+\S.*)$", re.MULTILINE)
    matches = list(pattern.finditer(text))
    chapters = []
    if not matches:
        chapters.append(("Chương 1", text.strip()))
    else:
        for i, match in enumerate(matches):
            chap_title = match.group(1).strip()
            start = match.start()
            end = matches[i+1].start() if (i+1) < len(matches) else len(text)
            chap_text = text[start:end].strip()
            chapters.append((chap_title, chap_text))
    return chapters

In [34]:
def split_into_clauses(chap_text):
    """
    Tách nội dung của một chương thành các điều dựa trên marker "Điều ...".
    Nếu không tìm thấy marker, toàn bộ nội dung chương được gán vào "Điều 1".
    """
    pattern = re.compile(r"^(Điều\s+\d+.*)$", re.MULTILINE)
    matches = list(pattern.finditer(chap_text))
    clauses = []
    if not matches:
        clauses.append(("Điều 1", chap_text.strip()))
    else:
        for i, match in enumerate(matches):
            clause_title = match.group(1).strip()
            start = match.start()
            end = matches[i+1].start() if (i+1) < len(matches) else len(chap_text)
            clause_text = chap_text[start:end].strip()
            clauses.append((clause_title, clause_text))
    return clauses


In [58]:
def save_text(text, output_path):
    """
    Lưu văn bản ra file text với encoding UTF-8.
    Đảm bảo rằng thư mục chứa file đã được tạo.
    """
    folder = os.path.dirname(output_path)
    os.makedirs(folder, exist_ok=True)
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(text)


In [60]:
def process_file(file_path, relative_path, output_folder):
    """
    Xử lý một file (PDF, DOCX, DOC):
      - Trích xuất văn bản từ file.
      - Tách văn bản thành các chương và trong mỗi chương tách ra các điều.
      - Lưu kết quả vào folder output_folder với cấu trúc:
            output_folder/relative_path/<tên file đã chuẩn hóa>/Chương_X/Điều_Y.txt
    """
    print(f"Đang xử lý file: {file_path}")
    
    if file_path.lower().endswith(".pdf"):
        full_text = extract_text_from_pdf(file_path)
    elif file_path.lower().endswith(".docx"):
        full_text = extract_text_from_docx(file_path)
    elif file_path.lower().endswith(".doc"):
        full_text = extract_text_from_doc(file_path)
    else:
        return

    chapters = split_into_chapters(full_text)
    base_name = os.path.splitext(os.path.basename(file_path))[0]
    base_output_dir = os.path.join(output_folder, relative_path, prepare_filename(base_name))
    os.makedirs(base_output_dir, exist_ok=True)

    for chap_title, chap_text in chapters:
        chap_folder_name = prepare_filename(chap_title)
        chapter_folder_path = os.path.join(base_output_dir, chap_folder_name)
        os.makedirs(chapter_folder_path, exist_ok=True)

        clauses = split_into_clauses(chap_text)
        for clause_title, clause_text in clauses:
            clause_file_name = prepare_filename(clause_title) + ".txt"
            output_file = os.path.join(chapter_folder_path, clause_file_name)
            save_text(clause_text, output_file)
            print(f"Đã lưu: {output_file}")

In [42]:
def process_file(file_path, relative_path, output_folder):
    """
    Xử lý một file (PDF hoặc DOCX):
      - Trích xuất văn bản.
      - Tách văn bản thành các chương và trong mỗi chương tách thành các điều.
      - Lưu kết quả ra folder mới "data" với cấu trúc:
          output_folder/relative_path/<tên file đã sanitize>/Chương_X/Điều_Y.txt
    """
    print(f"Đang xử lý file: {file_path}")
    
    # Trích xuất nội dung văn bản từ file
    if file_path.lower().endswith(".pdf"):
        full_text = extract_text_from_pdf(file_path)
    elif file_path.lower().endswith(".docx"):
        full_text = extract_text_from_docx(file_path)
    else:
        return

    # Tách nội dung thành các chương
    chapters = split_into_chapters(full_text)
    
    # Tạo folder cơ sở cho file hiện tại (sử dụng tên file đã được sanitize)
    base_name = os.path.splitext(os.path.basename(file_path))[0]
    base_output_dir = os.path.join(output_folder, relative_path, sanitize_name(base_name))
    os.makedirs(base_output_dir, exist_ok=True)

    # Duyệt qua các chương
    for chap_title, chap_text in chapters:
        chap_folder_name = sanitize_name(chap_title)
        chapter_folder_path = os.path.join(base_output_dir, chap_folder_name)
        os.makedirs(chapter_folder_path, exist_ok=True)

        # Tách chương thành các điều và lưu từng điều ra file text riêng
        clauses = split_into_clauses(chap_text)
        for clause_title, clause_text in clauses:
            clause_file_name = sanitize_name(clause_title) + ".txt"
            output_file = os.path.join(chapter_folder_path, clause_file_name)
            save_text(clause_text, output_file)
            print(f"Đã lưu: {output_file}")


In [62]:
def process_files(input_folder, output_folder):
    """
    Duyệt toàn bộ cây thư mục từ input_folder (ví dụ: "data_crawl")
    và xử lý các file PDF, DOCX, DOC. Kết quả sẽ được lưu vào folder output_folder,
    giữ nguyên cấu trúc thư mục con của input_folder.
    """
    for root, _, files in os.walk(input_folder):
        for file in files:
            if file.lower().endswith((".pdf", ".docx", ".doc")):
                file_path = os.path.join(root, file)
                relative_path = os.path.relpath(root, input_folder)
                process_file(file_path, relative_path, output_folder)


In [ ]:
input_folder = r"law_prediction-master\law_prediction-master\data_crawl" 
output_folder = "data_new"      
process_files(input_folder, output_folder)
print("Hoàn thành trích xuất và lưu dữ liệu vào folder 'data'.")

Đang xử lý file: law_prediction-master\law_prediction-master\data_crawl\bo_luat_dan_su\bo_luat_dan_su.pdf
Đã lưu: data_new\bo_luat_dan_su\bo_luat_dan_su\Chương I\Điều 1. Phạm vi điều chỉnh.txt
Đã lưu: data_new\bo_luat_dan_su\bo_luat_dan_su\Chương I\Điều 2. Công nhận, tôn trọng, bảo vệ và bảo đảm quyền dân sự.txt
Đã lưu: data_new\bo_luat_dan_su\bo_luat_dan_su\Chương I\Điều 3. Các nguyên tắc cơ bản của pháp luật dân sự.txt
Đã lưu: data_new\bo_luat_dan_su\bo_luat_dan_su\Chương I\Điều 4. Áp dụng Bộ luật dân sự.txt
Đã lưu: data_new\bo_luat_dan_su\bo_luat_dan_su\Chương I\Điều 5. Áp dụng tập quán.txt
Đã lưu: data_new\bo_luat_dan_su\bo_luat_dan_su\Chương I\Điều 6. Áp dụng tương tự pháp luật.txt
Đã lưu: data_new\bo_luat_dan_su\bo_luat_dan_su\Chương I\Điều 7. Chính sách của Nhà nước đối với quan hệ dân sự.txt
Đã lưu: data_new\bo_luat_dan_su\bo_luat_dan_su\Chương II\Điều 8. Căn cứ xác lập quyền dân sự.txt
Đã lưu: data_new\bo_luat_dan_su\bo_luat_dan_su\Chương II\Điều 9. Thực hiện quyền dân sự.txt
